In [ ]:
# Analysis of the output of experiments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os
import pathlib

import pandas as pd
import altair as alt
import numpy as np
import pvlib

import plotly.express as px

alt.data_transformers.disable_max_rows()


def _(df, *args, **kwargs):
    print(len(df))
    display(df.head(*args, **kwargs))

In [ ]:
# It's always annoying to set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

In [ ]:
%pwd

In [ ]:
def round_to(x, to):
    return round(x / to) * to

In [ ]:
EXP_NAMES = ["island-v3"]

REMOVE_WEIRD_PV_IDS = False

ROUND_HORIZON_TO = 4
ROUND_PRED_TO = 30
WIDTH = 120

In [ ]:
def load_models(names):
    dfs = []
    for name in names:
        try:
            df = pd.read_csv(f"exp_results/{name}/test_errors.csv")
        except FileNotFoundError as e:
            print(e)
            continue
        df["model"] = name
        df["mean_y"] = df["y"].mean()
        dfs.append(df)

    df = pd.concat(dfs)
    return df


def error_chart(names):

    df = load_models(names)
    #     _(df)

    df["ts"] = pd.to_datetime(df["ts"])
    df = df[df["metric"] == "mae"]

    #     _(df.sort_values(["ts", "horizon", "pv_id", "model"]), 200)

    df["pred_ts"] = df["ts"] + pd.to_timedelta(df["horizon"], unit="minute")

    # We have heuristics to remove weird test PVs that have data at night.
    # In the `uk_pv` dataset there are quite a few of those.
    # Note that this is pretty much hard-coded for that use-case.
    if REMOVE_WEIRD_PV_IDS:
        night_mask = (df["pred_ts"].dt.hour < 1) | (df["pred_ts"].dt.hour > 20)
        bad_pvs = df[night_mask & df["y"] > 0]
        bad_pvs = bad_pvs[["pv_id", "y"]].groupby("pv_id").count().reset_index()
        bad_pvs = bad_pvs[bad_pvs["y"] > 10]
        # _(bad_pvs)

        remove_pvs = bad_pvs["pv_id"].unique()

        print(f"REMOVING PVS WITH NIGHT DATA: {remove_pvs}")
        df = df[~df["pv_id"].isin(remove_pvs)]

    # df = df.join(inferred_meta[["factor", "capacity"]], on="pv_id")
    # df = df.join(meta[['latitude', 'longitude']], on="pv_id")

    # Normalizing by the average `y` values for each PV.
    df["weighted_error"] = df["error"] / df["mean_y"]

    df["horizon"] = df["horizon"] / 60.0
    df = df[~df["error"].isnull()]

    # Round the prediction hour and the horizon for more concise charts.
    df["pred_hour"] = df["pred_ts"].dt.hour * 60 + round_to(df["pred_ts"].dt.minute, ROUND_PRED_TO)
    df["horizon"] = round_to(df["horizon"], ROUND_HORIZON_TO)

    df = (
        df[["model", "horizon", "weighted_error", "pred_hour"]]
        .groupby(["model", "pred_hour", "horizon"])
        .mean()
        .reset_index()
    )

    df["pred_hour"] = pd.to_timedelta(df["pred_hour"], unit="minute")
    df["date"] = pd.Timestamp(2023, 1, 1)
    df["pred_hour"] = df["date"] + df["pred_hour"]

    mean_ = df.groupby(["model"])["weighted_error"].mean().to_dict()
    err = (
        df.groupby(["model"])["weighted_error"].std()
        * 1.96
        / np.sqrt(df.groupby(["model"])["weighted_error"].count())
    ).to_dict()

    print("Mean errors")
    for name in mean_:
        print(f"{name}: {mean_[name]:.4f} ± {err[name]:.4f}")

    chart = (
        alt.Chart(df)
        .mark_line()
        .encode(
            x=alt.X("hoursminutes(pred_hour)", title="Time *at prediction*"),
            y=alt.Y("weighted_error", title="Error"),
            color=alt.Color("model", sort=names),
            column=alt.Column("horizon:O", spacing=1),
        )
        .properties(
            height=WIDTH * 1.5,
            width=WIDTH,
        )
    )

    chart2 = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X("model"),
            y=alt.Y("mean(weighted_error)", title="Error"),
            color=alt.Color("model", sort=names),
            column=alt.Column("horizon:O"),
        )
        .properties(
            height=125,
            width=30,
        )
    )
    return chart, chart2


c1, c2 = error_chart(EXP_NAMES)
display(c1)
display(c2)

In [ ]:
df = load_models(EXP_NAMES)
# df["ts"] = pd.to_datetime(df["ts"])
# df = df[df["metric"] == "mae"]
# df = df.join(inferred_meta[["factor"]], on="pv_id")
# df = df.rename(columns={"factor": "capacity"})
# df["pred_ts"] = df["ts"] + df["horizon"].map(lambda x: pd.Timedelta(minutes=x))
# df["weighted_error"] = df["capacity"] * df["error"]
# df["ts_hour"] = df["ts"].dt.hour.astype(str) + ":" + df["ts"].dt.minute.astype(str)
# df["horizon"] = df["horizon"] / 60.0
# df = df[~df["error"].isnull()]
_(df)

scale = alt.Scale()  # clamp=True, domain=(0.01, 100))

max_ = df["y"].quantile(0.99)

chart = (
    alt.Chart(df.sample(1000))
    .mark_circle(opacity=0.5)
    .encode(
        x=alt.X("y", scale=scale, title="ground truth"),
        y=alt.Y("pred", scale=scale, title="Prediction"),
        color="model",
    )
    .properties(width=400, height=400)
) + (
    alt.Chart(pd.DataFrame(dict(x=[0, max_], y=[0, max_])))
    .mark_line(color="black")
    .encode(x="x", y="y")
)
chart